In [17]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
from scipy.optimize import root
from matplotlib.font_manager import FontProperties

rural = np.arange(0, 60, 2)
urban = np.arange(1, 60, 2)
east = np.r_[0, 1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 24, 25, 28, 29, 36, 37, 40, 41]
middle = np.r_[6, 7, 22, 23, 26, 27, 30, 31, 32, 33, 34, 35]
west = np.r_[8, 9, 10, 11, 12, 13, 14, 15, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
high_flow_in = [0, 1, 2, 3, 8, 10, 12, 18, 20] # highest 30%
low_flow_in = [4, 5, 6, 7, 9, 11, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29]

font_path = r'C:\Windows\Fonts\simsun.ttc'
chinese_font = FontProperties(fname=r'C:\Windows\Fonts\simsun.ttc')
english_font = FontProperties(fname=r'C:\Windows\Fonts\times.ttf')
pd.options.display.float_format = '{:.3f}'.format

def gini_index(array):
    cum_wealths = np.cumsum(sorted(np.append(array, 0)))
    sum_wealths = cum_wealths[-1]
    xarray = np.array(range(0, len(cum_wealths))) / float(len(cum_wealths)-1)
    yarray = cum_wealths / sum_wealths
    B = np.trapz(yarray, x=xarray)
    A = 0.5 - B
    return A / (A + B)


In [18]:
# import variables and parameters

equilibrium2010 = np.load('./data/equilibrium2010.npz')
R = equilibrium2010['R']
X = equilibrium2010['X']
m = equilibrium2010['m']
pi = equilibrium2010['pi']
L = equilibrium2010['L']
L_bar = equilibrium2010['L_bar']
K = equilibrium2010['K']
w = equilibrium2010['w']
P = equilibrium2010['P']
W = equilibrium2010['W']

parameter = np.load('./data/Parameter.npz')
kappa = parameter['kappa']
theta = parameter['theta']
beta = parameter['beta']
psi_ag = parameter['psi_ag']
psi_na = parameter['psi_na']
alpha_ag = parameter['alpha_ag']
alpha_na = parameter['alpha_na']
sigma_ag = parameter['sigma_ag']
sigma_na = parameter['sigma_na']
eta_ag_ag = parameter['eta_ag_ag']
eta_ag_na = parameter['eta_ag_na']
eta_na_ag = parameter['eta_na_ag']
eta_na_na = parameter['eta_na_na']
phiu = parameter['phiu']
phip = parameter['phip']
chi = parameter['chi']
eta = parameter['eta']
delta = parameter['delta']
xi_ag = parameter['xi_ag']
xi_na = parameter['xi_na']
t = parameter['t']

In [19]:
# import exogenous variables

K1017 = np.array(pd.read_excel('./data/raw_and_build/城镇村及工矿用地面积.xlsx').iloc[:, 3:])
K1017 = np.delete(K1017, 25, axis=0) # remove Xizang
dK1117 = K1017[:, 1:] / K1017[:, 0].reshape([30, 1])
dK1117 = K1017[:, 1:] / K1017[:, 0].reshape([30, 1])

dKh = np.concatenate([np.ones([30, 1]), dK1117[:, -1].reshape([30, 1])], axis=1).reshape([60, 1])
dKp = np.concatenate([np.ones([30, 1]), dK1117[:, -1].reshape([30, 1])], axis=1).reshape([60, 1])
dT = np.ones([60, 1])
dtau = np.ones([60, 60])
dmu = np.ones([60, 60])

In [20]:
# calculate the equlibrium without land reform

def SolveCounter_wo_reform(XX):
    dw = XX[:60].reshape([60, 1])
    dL = XX[60:120].reshape([60, 1])
    dp = XX[120:].reshape([60, 1])

    ## (a) change in price index
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))

    ## (b) change in public expenditures
    dG = dw * dL / dP

    ## (c) change in the cost of unit input bundle
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p

    ## (d) change in trade share
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta))
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) 
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi

    ## (e) change in the price of final goods
    dp1 = dpi_den ** (-1 / theta)

    ## (f) change in the income ratio (no change)
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    ddelta = np.ones([60, 60])
    
    ## (g) change in the migration share
    dg = dG / (dL ** chi) 
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa 
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) 
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm

    ## (h) change in the numebr of residents
    dL1 = (m_prime @ L_bar) / L

    ## (i) total outputs and expenditures under the equilibrium
    R_prime = R * dw * dL
    wL_prime = wL * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime 
    vL_prime = (wL_prime.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG_prime = (wL_prime.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A_prime = (beta * (1 - t) * np.repeat(vL_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C_prime = (np.repeat(PG_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag_prime = ((R_prime.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na_prime = ((R_prime.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_prime = np.concatenate([B_ag_prime, B_na_prime], axis=1).reshape([60, 1])
    X_prime1 = A_prime + B_prime + C_prime 

    ## (j) conditions for the equilibrium
    res1 = dp1 - dp
    res2 = dL1 - dL
    res3 = X_prime1 - X_prime

    return np.concatenate([res1, res2, res3]).flatten().astype(float)

In [21]:
# calculate the equlibrium with land reform

def SolveCounter(XX):
    dw = XX[:60].reshape([60, 1])
    dL = XX[60:120].reshape([60, 1])
    dp = XX[120:].reshape([60, 1])

    ## (a) change in price index
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))

    ## (b) change in public expenditures
    dG = dw * dL / dP

    ## (c) change in the cost of the unit input bundle
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p

    ## (d) change in the trade share
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) 
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) 
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi 

    ## (e) change in the price of final goods
    dp1 = dpi_den ** (-1 / theta)

    ## (f) change in the income ratio
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1]) 
    w = wL / L 
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1]) 
    rho_prime = rKrK * dw * dL / L_bar
    rho_prime[1::2] = 0 
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T 
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) 
    ddelta = delta_prime
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) 

    ## (g) change in the migration share
    dg = dG / (dL ** chi)
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa 
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) 
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm

    ## (h) change in the number of residents
    dL1 = (m_prime @ L_bar) / L

    ## (i) total outputs and expenditures under the equilibrium
    R_prime = R * dw * dL
    wL_prime = wL * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime 
    vL_prime = (wL_prime.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG_prime = (wL_prime.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A_prime = (beta * (1 - t) * np.repeat(vL_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C_prime = (np.repeat(PG_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag_prime = ((R_prime.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na_prime = ((R_prime.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_prime = np.concatenate([B_ag_prime, B_na_prime], axis=1).reshape([60, 1])
    X_prime1 = A_prime + B_prime + C_prime 

    ## (j) conditions for the equilibrium
    res1 = dp1 - dp
    res2 = dL1 - dL
    res3 = X_prime1 - X_prime

    return np.concatenate([res1, res2, res3]).flatten().astype(float)

In [22]:
# save variables without the land reform

def counterfautual_wo_reform_result(dw, dL, dp):
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))
    dG = dw * dL / dP
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) 
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1])
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi 
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    w = wL / L 
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1])
    ddelta = np.ones([60, 60])
    dg = dG / (dL ** chi)
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa 
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) 
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm
    R_prime = R * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime 

    return dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag

In [23]:
# save variables with the land reform

def counterfautual_result(dw, dL, dp):
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))
    dG = dw * dL / dP
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) 
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1])
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi 
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    w = wL / L 
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1]) 
    rho_prime = rKrK * dw * dL / L_bar
    rho_prime[1::2] = 0 
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T 
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) 
    ddelta = delta_prime
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) 
    dg = dG / (dL ** chi)
    dW = dw / dP 
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa 
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) 
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm
    R_prime = R * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime 

    return dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag

In [24]:
# define the function to calculate welfare

def welfare():
    coef_g_na = 1 - beta + sigma_na / alpha_na + beta * t
    coef_g_ag = (alpha_ag + sigma_ag) * t / (beta + t - beta * t) / alpha_ag
    g = ((w * L).reshape([30, 2]) * np.array([coef_g_ag, coef_g_na]).reshape([1, 2])).reshape([60, 1]) / P / (L ** chi)
    
    dV = dW * ((dg) ** phiu) * (np.diag(dm).reshape([-1, 1]) ** (-1 / kappa)) * (np.diag(ddelta).reshape([-1, 1]))

    weight = (np.diag(m).reshape([-1, 1]) ** (-1 / kappa)) * (g ** phiu) * W * L_bar * delta_diag
    weight_country = weight / weight.sum()
    weight_urban = weight[urban] / weight[urban].sum()
    weight_rural = weight[rural] / weight[rural].sum()
    weight_east = weight[east] / weight[east].sum()
    weight_middle = weight[middle] / weight[middle].sum()
    weight_west = weight[west] / weight[west].sum()

    dV_country = ((dV * weight_country).sum() - 1) * 100
    dV_urban = ((dV[urban] * weight_urban).sum() - 1) * 100
    dV_rural = ((dV[rural] * weight_rural).sum() - 1) * 100
    dV_east = ((dV[east] * weight_east).sum() - 1) * 100
    dV_middle = ((dV[middle] * weight_middle).sum() - 1) * 100
    dV_west = ((dV[west] * weight_west).sum() - 1) * 100
    
    return dV, dV_country, dV_urban, dV_rural, dV_east, dV_middle, dV_west

In [25]:
# variables under baseline equilibrium

W = w / P
W_country2010 = (W * L)[:-1].sum() / L[:-1].sum()
W_urban2010 = (W * L)[urban].sum() / L[urban].sum()
W_rural2010 = (W * L)[rural].sum() / L[rural].sum()
incgini_country2010 = gini_index(W)
incgini_urban2010 = gini_index(W[urban])
incgini_rural2010 = gini_index(W[rural])
incgini_urbanrural2010 = gini_index([W[urban].mean(), W[rural].mean()])
migrant2010 = L[:-1].sum() - np.diag((m.T * L_bar).T)[:-1].sum()
L_urban2010 = L[urban].sum()
L_rural2010 = L[rural].sum()

W_country2010, W_urban2010, W_rural2010
incgini_country2010, incgini_urban2010, incgini_rural2010, incgini_urbanrural2010

(0.313447120802688, 0.147606995653925, 0.2073539013397404, 0.23130855681338192)

In [26]:
# equilibrium: no land reform, uniform land supply

W_prime_country_avgsupply, W_prime_urban_avgsupply, W_prime_rural_avgsupply = [], [], []
dV_country_avgsupply, dV_urban_avgsupply, dV_rural_avgsupply = [], [], []
gini_index_country_avgsupply, gini_index_urban_avgsupply, gini_index_rural_avgsupply, gini_index_urbanrural_avgsupply = [], [], [], []
migrant_prime_country_avgsupply = []
L_prime_urban_avgsupply, L_prime_rural_avgsupply = [], []

for year in range(2011, 2018, 1):
    dK_total = K1017[:, year-2010].sum() / K1017[:, 0].sum()
    dKp = np.concatenate([np.ones([30, 1]), np.ones([30, 1]) * dK_total], axis=1).reshape([60, 1])
    dKh = np.concatenate([np.ones([30, 1]), np.ones([30, 1]) * dK_total], axis=1).reshape([60, 1])
    XX = root(SolveCounter_wo_reform, x0=np.ones(3*60), method='lm') 
    dw = XX.x[:60].reshape([60, 1])
    dL = XX.x[60:120].reshape([60, 1])
    dp = XX.x[120:].reshape([60, 1])
    dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag = counterfautual_wo_reform_result(dw, dL, dp)
    W = w / P
    w_prime = w * dw
    L_prime = L * dL
    P_prime = P * dP
    W_prime = w_prime / P_prime

    ## real gdp per capita
    each_year_W_prime_country = (w_prime * L_prime / P_prime).sum() / L_prime.sum()
    each_year_W_prime_urban = (w_prime * L_prime / P_prime)[urban].sum() / L_prime[urban].sum()
    each_year_W_prime_rural = (w_prime * L_prime / P_prime)[rural].sum() / L_prime[rural].sum()
    W_prime_country_avgsupply.append(each_year_W_prime_country)
    W_prime_urban_avgsupply.append(each_year_W_prime_urban)
    W_prime_rural_avgsupply.append(each_year_W_prime_rural)

    ## welfare
    each_year_dV, each_year_dV_country, each_year_dV_urban, each_year_dV_rural, each_year_dV_east, each_year_dV_middle, each_year_dV_west = welfare()
    dV_country_avgsupply.append(each_year_dV_country)
    dV_urban_avgsupply.append(each_year_dV_urban)
    dV_rural_avgsupply.append(each_year_dV_rural)

    ## gini coefficients for real income
    each_year_incgini_country = gini_index(W_prime)
    each_year_incgini_urban = gini_index(W_prime[urban])
    each_year_incgini_rural = gini_index(W_prime[rural])
    each_year_incgini_urbanrural = gini_index([W_prime[urban].mean(), W_prime[rural].mean()])
    gini_index_country_avgsupply.append(each_year_incgini_country)
    gini_index_urban_avgsupply.append(each_year_incgini_urban)
    gini_index_rural_avgsupply.append(each_year_incgini_rural)
    gini_index_urbanrural_avgsupply.append(each_year_incgini_urbanrural)
    
    ## total number of migrants
    each_migrant = L_prime[:-1].sum() - np.diag((m_prime.T * L_bar).T)[:-1].sum()
    migrant_prime_country_avgsupply.append(each_migrant)
    
    ## total number of residents
    each_L_prime_urban = L_prime[urban].sum()
    each_L_prime_rural = L_prime[rural].sum()
    L_prime_urban_avgsupply.append(each_L_prime_urban)
    L_prime_rural_avgsupply.append(each_L_prime_rural)


In [27]:
# equilibrium: no land reform, real land supply

W_prime_country_supply, W_prime_urban_supply, W_prime_rural_supply = [], [], []
dV_country_supply, dV_urban_supply, dV_rural_supply = [], [], []
gini_index_country_supply, gini_index_urban_supply, gini_index_rural_supply, gini_index_urbanrural_supply = [], [], [], []
migrant_prime_country_supply = []
L_prime_urban_supply, L_prime_rural_supply = [], []

for year in range(2011, 2018, 1):
    dKp = np.concatenate([np.ones([30, 1]), dK1117[:, year-2011].reshape([30, 1])], axis=1).reshape([60, 1])
    dKh = np.concatenate([np.ones([30, 1]), dK1117[:, year-2011].reshape([30, 1])], axis=1).reshape([60, 1])
    XX = root(SolveCounter_wo_reform, x0=np.ones(3*60), method='lm') 
    dw = XX.x[:60].reshape([60, 1])
    dL = XX.x[60:120].reshape([60, 1])
    dp = XX.x[120:].reshape([60, 1])
    dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag = counterfautual_wo_reform_result(dw, dL, dp)
    W = w / P
    w_prime = w * dw
    L_prime = L * dL
    P_prime = P * dP
    W_prime = w_prime / P_prime

    ## real gdp per capita
    each_year_W_prime_country = (w_prime * L_prime / P_prime).sum() / L_prime.sum()
    each_year_W_prime_urban = (w_prime * L_prime / P_prime)[urban].sum() / L_prime[urban].sum()
    each_year_W_prime_rural = (w_prime * L_prime / P_prime)[rural].sum() / L_prime[rural].sum()
    W_prime_country_supply.append(each_year_W_prime_country)
    W_prime_urban_supply.append(each_year_W_prime_urban)
    W_prime_rural_supply.append(each_year_W_prime_rural)

    ## welfare
    each_year_dV, each_year_dV_country, each_year_dV_urban, each_year_dV_rural, each_year_dV_east, each_year_dV_middle, each_year_dV_west = welfare()
    dV_country_supply.append(each_year_dV_country)
    dV_urban_supply.append(each_year_dV_urban)
    dV_rural_supply.append(each_year_dV_rural)

    ## gini coefficients for real gdp per capita
    each_year_incgini_country = gini_index(W_prime)
    each_year_incgini_urban = gini_index(W_prime[urban])
    each_year_incgini_rural = gini_index(W_prime[rural])
    each_year_incgini_urbanrural = gini_index([W_prime[urban].mean(), W_prime[rural].mean()])
    gini_index_country_supply.append(each_year_incgini_country)
    gini_index_urban_supply.append(each_year_incgini_urban)
    gini_index_rural_supply.append(each_year_incgini_rural)
    gini_index_urbanrural_supply.append(each_year_incgini_urbanrural)

    ## total number of migrants
    each_migrant = L_prime[:-1].sum() - np.diag((m_prime.T * L_bar).T)[:-1].sum()
    migrant_prime_country_supply.append(each_migrant)
    
    ## total number of residents
    each_L_prime_urban = L_prime[urban].sum()
    each_L_prime_rural = L_prime[rural].sum()
    L_prime_urban_supply.append(each_L_prime_urban)
    L_prime_rural_supply.append(each_L_prime_rural)


In [28]:
# equilibrium: land reform, real land supply

W_prime_country_both, W_prime_urban_both, W_prime_rural_both = [], [], []
dV_country_both, dV_urban_both, dV_rural_both = [], [], []
gini_index_country_both, gini_index_urban_both, gini_index_rural_both, gini_index_urbanrural_both = [], [], [], []
migrant_prime_country_both = []
L_prime_urban_both, L_prime_rural_both = [], []

for year in range(2011, 2018, 1):
    dKp = np.concatenate([np.ones([30, 1]), dK1117[:, year-2011].reshape([30, 1])], axis=1).reshape([60, 1])
    dKh = np.concatenate([np.ones([30, 1]), dK1117[:, year-2011].reshape([30, 1])], axis=1).reshape([60, 1])
    XX = root(SolveCounter, x0=np.ones(3*60), method='lm') # 求解非线性方程组
    dw = XX.x[:60].reshape([60, 1])
    dL = XX.x[60:120].reshape([60, 1])
    dp = XX.x[120:].reshape([60, 1])
    dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag = counterfautual_result(dw, dL, dp)
    W = w / P
    w_prime = w * dw
    L_prime = L * dL
    P_prime = P * dP
    W_prime = w_prime / P_prime

    ## real gdp per capita
    each_year_W_prime_country = (w_prime * L_prime / P_prime).sum() / L_prime.sum()
    each_year_W_prime_urban = (w_prime * L_prime / P_prime)[urban].sum() / L_prime[urban].sum()
    each_year_W_prime_rural = (w_prime * L_prime / P_prime)[rural].sum() / L_prime[rural].sum()
    W_prime_country_both.append(each_year_W_prime_country)
    W_prime_urban_both.append(each_year_W_prime_urban)
    W_prime_rural_both.append(each_year_W_prime_rural)

    ## welfare
    each_year_dV, each_year_dV_country, each_year_dV_urban, each_year_dV_rural, each_year_dV_east, each_year_dV_middle, each_year_dV_west = welfare()
    dV_country_both.append(each_year_dV_country)
    dV_urban_both.append(each_year_dV_urban)
    dV_rural_both.append(each_year_dV_rural)

    ## gini coefficients for real gdp per capita
    each_year_incgini_country = gini_index(W_prime)
    each_year_incgini_urban = gini_index(W_prime[urban])
    each_year_incgini_rural = gini_index(W_prime[rural])
    each_year_incgini_urbanrural = gini_index([W_prime[urban].mean(), W_prime[rural].mean()])
    gini_index_country_both.append(each_year_incgini_country)
    gini_index_urban_both.append(each_year_incgini_urban)
    gini_index_rural_both.append(each_year_incgini_rural)
    gini_index_urbanrural_both.append(each_year_incgini_urbanrural)

    ## total number of migrants
    each_migrant = L_prime[:-1].sum() - np.diag((m_prime.T * L_bar).T)[:-1].sum()
    migrant_prime_country_both.append(each_migrant)
    
    ## total number of residents
    each_L_prime_urban = L_prime[urban].sum()
    each_L_prime_rural = L_prime[rural].sum()
    L_prime_urban_both.append(each_L_prime_urban)
    L_prime_rural_both.append(each_L_prime_rural)


In [29]:
# summary

W_prime_country_compare = pd.concat([
    pd.Series([W_country2010] + W_prime_country_avgsupply), 
    pd.Series([W_country2010] + W_prime_country_supply), 
    pd.Series([W_country2010] + W_prime_country_both)
    ], axis=1
)

V_prime_country_compare = pd.concat([
    1 + pd.Series([0] + dV_country_avgsupply) / 100, 
    1 + pd.Series([0] + dV_country_supply) / 100, 
    1 + pd.Series([0] + dV_country_both) / 100
    ], axis=1
)

incgini_country_compare = pd.concat([
    pd.Series([incgini_country2010] + gini_index_country_avgsupply), 
    pd.Series([incgini_country2010] + gini_index_country_supply), 
    pd.Series([incgini_country2010] + gini_index_country_both)
    ], axis=1
)

incgini_urban_compare = pd.concat([
    pd.Series([incgini_urban2010] + gini_index_urban_avgsupply), 
    pd.Series([incgini_urban2010] + gini_index_urban_supply), 
    pd.Series([incgini_urban2010] + gini_index_urban_both)
    ], axis=1
)

incgini_rural_compare = pd.concat([
    pd.Series([incgini_rural2010] + gini_index_rural_avgsupply), 
    pd.Series([incgini_rural2010] + gini_index_rural_supply), 
    pd.Series([incgini_rural2010] + gini_index_rural_both)
    ], axis=1
)

incgini_urbanrural_compare = pd.concat([
    pd.Series([incgini_urbanrural2010] + gini_index_urbanrural_avgsupply), 
    pd.Series([incgini_urbanrural2010] + gini_index_urbanrural_supply), 
    pd.Series([incgini_urbanrural2010] + gini_index_urbanrural_both)
    ], axis=1
)

migrant_prime_country_compare = pd.concat([
    pd.Series([migrant2010] + migrant_prime_country_avgsupply), 
    pd.Series([migrant2010] + migrant_prime_country_supply), 
    pd.Series([migrant2010] + migrant_prime_country_both)
    ], axis=1
)

L_prime_urban_compare = pd.concat([
    pd.Series([L_urban2010] + L_prime_urban_avgsupply), 
    pd.Series([L_urban2010] + L_prime_urban_supply), 
    pd.Series([L_urban2010] + L_prime_urban_both)
    ], axis=1
)


L_prime_rural_compare = pd.concat([
    pd.Series([L_rural2010] + L_prime_rural_avgsupply), 
    pd.Series([L_rural2010] + L_prime_rural_supply), 
    pd.Series([L_rural2010] + L_prime_rural_both)
    ], axis=1
)


In [30]:
# comparison (Table 5)

dmigrant_compare2017 = (migrant_prime_country_compare.iloc[-1, :] / migrant_prime_country_compare.iloc[0, :] - 1) * 100
dL_urban_compare2017 = (L_prime_urban_compare.iloc[-1, :] / L_prime_urban_compare.iloc[0, :] - 1) * 100
dL_rural_compare2017 = (L_prime_rural_compare.iloc[-1, :] / L_prime_rural_compare.iloc[0, :] - 1) * 100
dW_country_compare2017 = (W_prime_country_compare.iloc[-1, :] / W_prime_country_compare.iloc[0, :] - 1) * 100
dV_country_compare2017 = (V_prime_country_compare.iloc[-1, :] / V_prime_country_compare.iloc[0, :] - 1) * 100
dincgini_country_compare2017 = (incgini_country_compare.iloc[-1, :] / incgini_country_compare.iloc[0, :] - 1) * 100
dincgini_urban_compare2017 = (incgini_urban_compare.iloc[-1, :] / incgini_urban_compare.iloc[0, :] - 1) * 100
dincgini_rural_compare2017 = (incgini_rural_compare.iloc[-1, :] / incgini_rural_compare.iloc[0, :] - 1) * 100
dincgini_urbanrural_compare2017 = (incgini_urbanrural_compare.iloc[-1, :] / incgini_urbanrural_compare.iloc[0, :] - 1) * 100

country_compare2017 = pd.concat(
    [dL_urban_compare2017, dL_rural_compare2017, dmigrant_compare2017, dW_country_compare2017, dV_country_compare2017, dincgini_country_compare2017, dincgini_urban_compare2017, dincgini_rural_compare2017, dincgini_urbanrural_compare2017], 
axis=1).T
country_compare2017.index = ['urban labor', 'rural labor', 'total migrants', 'real gdp per capita', 'welfare', 'gini coef. for real gdp per capita', 'gini coef. in the urban sector', 'gini coef. in the rural sector', 'urban-rural inequality']
country_compare2017.columns = ['Uniform allocations', 'Real‑world allocations', 'Real‑world allocations + land tilting reform']
country_compare2017

Uniform allocations  \
urban labor                                       0.177   
rural labor                                      -0.260   
total migrants                                    0.277   
real gdp per capita                               2.177   
welfare                                           2.199   
gini coef. for real gdp per capita                0.292   
gini coef. in the urban sector                    0.003   
gini coef. in the rural sector                    0.145   
urban-rural inequality                            0.398   

                                    Real‑world allocations  \
urban labor                                          0.192   
rural labor                                         -0.282   
total migrants                                       0.268   
real gdp per capita                                  2.073   
welfare                                              2.110   
gini coef. for real gdp per capita                  -0.005   
gini coef. in the urban sector                      -1.619   
gini coef. in the rural sector                      -0.405   
urban-rural inequality                               0.480   

                                    Real‑world allocations + land tilting reform  
urban labor                                                                7.208  
rural labor                                                              -10.573  
total migrants                                                            28.351  
real gdp per capita                                                        6.403  
welfare                                                                    3.814  
gini coef. for real gdp per capita                                        -9.443  
gini coef. in the urban sector                                            -5.003  
gini coef. in the rural sector                                            12.692  
urban-rural inequality                                                   -15.942